In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
datagen = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.xception.preprocess_input,
    validation_split=0.2,
)

In [3]:
train_gen = datagen.flow_from_directory(
    '../cub/CUB_200_2011/images',
    target_size=(299,299),
    batch_size=50,
    subset='training',
    shuffle=True,
)

validation_gen = datagen.flow_from_directory(
    '../cub/CUB_200_2011/images',
    target_size=(299,299),
    batch_size=50,
    subset='validation',
    shuffle=True,
)


Found 9465 images belonging to 200 classes.
Found 2323 images belonging to 200 classes.


In [4]:
train_gen.next()[0].shape

(50, 299, 299, 3)

In [17]:
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Activation, Flatten, Dropout, BatchNormalization

model = tf.keras.models.Sequential()
model.add(tf.keras.applications.Xception(include_top=False, pooling='avg', weights="imagenet"))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='softmax'))
model.layers[0].trainable = False


In [18]:
%%time
model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

CPU times: user 3.68 ms, sys: 50 µs, total: 3.73 ms
Wall time: 3.47 ms


In [ ]:
%%time
history = model.fit(
    train_gen,
    epochs=100,
    validation_data=validation_gen,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            min_delta=0.001,
            mode='max',
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath='checkpoints3',
            save_weights_only=True,
        ),
    ]
)

Epoch 1/100
190/190 [==============================] - 764s 4s/step - loss: 3.8072 - accuracy: 0.1704 - val_loss: 2.3518 - val_accuracy: 0.4335
Epoch 2/100
190/190 [==============================] - 760s 4s/step - loss: 2.2752 - accuracy: 0.3970 - val_loss: 1.7612 - val_accuracy: 0.5437
Epoch 3/100
190/190 [==============================] - 761s 4s/step - loss: 1.8356 - accuracy: 0.4973 - val_loss: 1.5353 - val_accuracy: 0.5880
Epoch 4/100
190/190 [==============================] - 762s 4s/step - loss: 1.5749 - accuracy: 0.5659 - val_loss: 1.4153 - val_accuracy: 0.6113
Epoch 5/100
190/190 [==============================] - 760s 4s/step - loss: 1.3982 - accuracy: 0.6038 - val_loss: 1.3186 - val_accuracy: 0.6328
Epoch 6/100
190/190 [==============================] - 762s 4s/step - loss: 1.2481 - accuracy: 0.6378 - val_loss: 1.2807 - val_accuracy: 0.6505
Epoch 7/100
190/190 [==============================] - 761s 4s/step - loss: 1.1500 - accuracy: 0.6654 - val_loss: 1.2807 - val_accuracy:

In [ ]:
%%time
model.evaluate(validation_gen)

In [ ]:
import os
model.save(f'model.{os.uname()[1]}.003')

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()